In [1]:
%matplotlib widget
import numpy as np
from astropy import units as u, constants as c
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18 as cosmo
import matplotlib
from astropy.convolution import convolve, Gaussian2DKernel
from scipy.optimize import fsolve, fmin
from astropy.io import fits
import scipy.interpolate as interp

In [2]:
cnfw = 3.3 # NFW concentration parameter
z = 2.3 # redshift
Rs = 27 # kpc = Rvir = 90 kpc
H = cosmo.H(z).to('km/s/kpc').value

noise_level = 0.01
# print(H)
# M_h = 1e12*u.M_sun
# rho0 = 3*M_h/(4*np.pi*Rs**3) * 0.5
# A = 8*np.pi*c.G*rho0*Rs**3/(1*u.kpc)
A = 1.2e7 #* u.km**2/u.s**2

bres = 2 # kpc
min_b = 10 # kpc
max_b = 250 # kpc
bvec = np.arange(min_b, max_b, bres)
vrawsamp = np.arange(-1000, 1015, 30)

bm, vm = np.meshgrid(bvec, vrawsamp)

bvec_final = np.arange(min_b, max_b+2.5, 5)
vvec_final = np.arange(-1000, 1050, 100)

In [3]:
def vins(r, voff):
    vin = voff + H*r
    return vin

def vouts(r, vi):
    vout = np.sqrt(vi**2 + A*(-np.log((Rs+1)/Rs) + (1/r * np.log((Rs+r)/Rs))))
    return vout
    
def ains(r, a0in, gin):
    ain = (a0in * (r/100)**(-gin))
    return ain

def aouts(r, a0out, gout):
    aout = (a0out * (r/100)**(-gout))
    return aout

In [4]:
def model(theta):
    vi, voff, a0out, a0in, gout, gin = theta

    rmax_out = 100 * (a0out/noise_level)**(1/gout)
    print(rmax_out)
    rmax_in_est = 100 * (a0in/noise_level)**(1/gin)
    rmax_in_max = 10000
    rmax_in = np.min([rmax_in_est, rmax_in_max])
    print(rmax_in)


    rawhmap = np.zeros((len(vrawsamp), len(bvec)))

    
    
    for bi, b in enumerate(bvec):

        lmax_out = np.sqrt(rmax_out**2 - b**2)
        lmax_in = np.sqrt(rmax_in**2 - b**2)


        larr_slowl_out = np.linspace(-lmax_out, -100, 100)
        nfast_out = int(1500 * np.exp(-b/100)) #1500
        larr_fast_out = np.linspace(-100, 100, nfast_out)
        larr_slowr_out = np.linspace(100, lmax_out, 100)
        larr_out = np.concatenate([larr_slowl_out, larr_fast_out, larr_slowr_out])
        r_out = np.sqrt(larr_out**2 + b**2)
        
        tau_outs = aouts(r_out, a0out, gout)
        tau_outs_arr = np.array(tau_outs)


        larr_slowl_in = np.linspace(-lmax_in, -100, 100)
        nfast_in = int(1500 * np.exp(-b/100)) #1500
        larr_fast_in = np.linspace(-100, 100, nfast_in)
        larr_slowr_in = np.linspace(100, lmax_in, 100)
        larr_in = np.concatenate([larr_slowl_in, larr_fast_in, larr_slowr_in])
        r_in = np.sqrt(larr_in**2 + b**2)
        
        tau_ins = ains(r_in, a0in, gin)
        tau_ins_arr = np.array(tau_ins)


        vLOS_out = larr_out/r_out*vouts(r_out, vi)
        vLOS_in = larr_in/r_in*vins(r_in, voff)
        # plt.figure()
        # sc = plt.scatter(larr_out, vLOS_out, c=tau_outs)
        # sc1 = plt.scatter(larr_in, vLOS_in, c=tau_ins, cmap='inferno')
        # plt.xlabel('$\\ell$ (kpc)')
        # plt.ylabel('$v_\\mathrm{LOS}$ (km/s)')
        # plt.colorbar(sc, label = '$\\tau_\\mathrm{out}$')
        # plt.colorbar(sc1, label = '$\\tau_\\mathrm{in}$')
        # plt.title(f'$b={b}$ kpc')
        # plt.axhline(0, c = 'k', alpha = 0.7, lw = 0.7, ls = '--')
        # plt.axvline(0, c = 'k', alpha = 0.7, lw = 0.7, ls = '--')
        # plt.tight_layout()
        # plt.show()
    
        maxvout = np.nanmax(vLOS_out)
        maxvin = np.nanmax(vLOS_in)

        # bad b's
        if ~np.isfinite(maxvout):
            for j in vrawsamp:
                taulist_out.append([j, 0])
            continue
            
        if ~np.isfinite(maxvin):
            # print(f'bad {b}')
            for j in vrawsamp:
                taulist_in.append([j, 0])
            continue
        
        l_minvout = larr_out[np.argmin(vLOS_out)]
        l_maxvout = larr_out[np.argmax(vLOS_out)]

        l_minvin = larr_in[np.argmin(vLOS_in)]
        l_maxvin = larr_in[np.argmax(vLOS_in)]


        taulist_out = []
        taulist_in = []
        
        for vl in vrawsamp:
            # outflow
            if np.abs(vl) > maxvout:
                taulist_out.append([vl+15, 0])
            else:
                possible_values = larr_out[(vLOS_out > vl) & (vLOS_out < vl+30)]
                l_near = possible_values[(possible_values > l_minvout) & (possible_values < l_maxvout)]
                l_far = possible_values[(possible_values < l_minvout) | (possible_values > l_maxvout)]
        
                tau_near_inds = [np.argwhere(larr_out == ln)[0][0] for ln in l_near]
                tau_out_near = tau_outs_arr[tau_near_inds]
        
                
                tau_far_inds = [np.argwhere(larr_out == ln)[0][0] for ln in l_far]
                tau_out_far = tau_outs_arr[tau_far_inds]
                tau_tot_out = np.nanmean(tau_out_near) + np.nan_to_num(np.nanmean(tau_out_far))
    
                if np.isfinite(tau_tot_out):
                    taulist_out.append([vl+15, tau_tot_out])

            # inflow
            if np.abs(vl) > maxvin:
                taulist_in.append([vl+15, 0])
            else:
                possible_values = larr_in[(vLOS_in > vl) & (vLOS_in < vl+30)]
                # print(possible_values)
                l_near = possible_values[(possible_values < l_minvin) & (possible_values > l_maxvin)]
                l_far = possible_values[(possible_values > l_minvin) | (possible_values < l_maxvin)]
        
                tau_near_inds = [np.argwhere(larr_in == ln)[0][0] for ln in l_near]
                tau_in_near = tau_ins_arr[tau_near_inds]
        
                
                tau_far_inds = [np.argwhere(larr_in == ln)[0][0] for ln in l_far]
                tau_in_far = tau_ins_arr[tau_far_inds]
                tau_tot_in = np.nan_to_num(np.nanmean(tau_in_near)) + np.nan_to_num(np.nanmean(tau_in_far))
                # print(tau_tot)
    
                if np.isfinite(tau_tot_in):
                    taulist_in.append([vl+15, tau_tot_in])
        
        tauarr_out = np.array(taulist_out) # v, tau
        tauarr_in = np.array(taulist_in)

        if len(tauarr_out) == 0:
            tout = vrawsamp*0.0
        else:
            tout = np.interp(vrawsamp, tauarr_out[:,0], tauarr_out[:,1], left=0, right=0)

        if len(tauarr_in) == 0:
            tin = vrawsamp*0.0
        else:
            tin = np.interp(vrawsamp, tauarr_in[:,0], tauarr_in[:,1], left=0, right=0)

        rawhmap[:,bi] = tout + tin
            # tau_tot = tau_tot_out + tau_tot_in
            # taulist.append([b, vl+15, tau_tot])

    
    

    # points = tauarr[:,0:2]
    # values = tauarr[:,2]
    # rawhmap = interp.griddata(points, values, (bm, vm), method='nearest')

    plt.figure()
    plt.imshow(rawhmap, origin='lower')
    plt.show()


    
    
    # print(tauarr_out[np.isfinite(tauarr_out[:,2])])
    # plt.figure()
    # plt.scatter(tauarr_out[:,1], tauarr_out[:,2])
    # plt.plot(vrawsamp, tout)
    # plt.plot(vrawsamp, tin)
    # plt.show()
    
    # plt.figure()
    # sc = plt.scatter(tauarr_out[:,0], tauarr_out[:,1], c=tauarr_out[:,2]) #/cmap='inferno_r' vmin=2e-2, vmax = 1e-1 norm=matplotlib.colors.LogNorm()
    # plt.colorbar(sc, label = '$\\tau_\\mathrm{out}$')
    
    # # tin = plt.scatter(tauarr_in[:,0], tauarr_in[:,1], c=tauarr_in[:,2], cmap='plasma_r') #/cmap='inferno_r' vmin=2e-2, vmax = 1e-1 norm=matplotlib.colors.LogNorm()
    # # plt.colorbar(tin, label = '$\\tau_\\mathrm{in}$')
    # plt.xlabel('b (kpc)')
    # plt.ylabel('$v_\\mathrm{LOS}$ (km/s)')
    # plt.tight_layout()
    # plt.show()
    
    # make map
    
    # vres = 10 #50
    # vlosr = np.arange(-1000, 1000, vres)
    
    # hmap = np.zeros((len(vlosr), len(bvec)))
    
    # for bi, b in enumerate(bvec):
    #     bcut_out = (tauarr_out[:,0] == b)
    #     bcut_in = (tauarr_in[:,0] == b)
    
    #     for vind, v in enumerate(vlosr):
    #         vcut_out = (tauarr_out[:,1] > v) & (tauarr_out[:,1] < v+vres)
    #         vcut_in = (tauarr_in[:,1] > v) & (tauarr_in[:,1] < v+vres)
            
    #         tauavg_out = np.nanmean(tauarr_out[:,2][bcut_out & vcut_out])
    #         if ~np.isfinite(tauavg_out):
    #             tauavg_out = 0
    
    #         tauavg_in = np.nanmean(tauarr_in[:,2][bcut_in & vcut_in])
    #         if ~np.isfinite(tauavg_in):
    #             tauavg_in = 0

    #         hmap[vind,bi] = tauavg_out + tauavg_in

    hmap_conv = convolve(rawhmap, Gaussian2DKernel(2,1), boundary = 'extend') # 100 km/s and 5 kpc sampling

    f = interp.RectBivariateSpline(vrawsamp, bvec, hmap_conv, kx=3, ky=3)
    hmap_reshaped = f(vvec_final, bvec_final)
    
    # print(new_im.shape)
    
    plt.figure()
    plt.imshow(hmap_reshaped, aspect='auto', origin='lower', extent = [bvec_final.min(), bvec_final.max(), vvec_final.min(), vvec_final.max()])
    plt.show()
    
    # # print(hmap.shape)
    plt.figure()
    us = plt.imshow(rawhmap, origin='lower', extent = (bvec[0], bvec[-1], vrawsamp[0], vrawsamp[-1]), 
               aspect='auto', cmap='plasma')
    plt.colorbar(us, label = '$\\tau_\\mathrm{tot}$')
    plt.xlabel('b (kpc)')
    plt.ylabel('$v_\\mathrm{LOS}$ (km/s)')
    plt.title('Unsmoothed')
    # plt.xscale('log')
    plt.show()
    
    plt.figure()
    sm = plt.imshow(hmap_conv, origin='lower', extent = (bvec[0], bvec[-1], vrawsamp[0], vrawsamp[-1]), 
               aspect='auto', cmap='inferno')
    plt.colorbar(sm, label = '$\\tau_\\mathrm{tot}$')
    plt.xlabel('b (kpc)')
    plt.ylabel('$v_\\mathrm{LOS}$ (km/s)')
    plt.title('Smoothed')
    # plt.xscale('log')
    plt.show()

    return hmap_reshaped
    


In [5]:
lya_real = fits.open('lya_conv_240623.fits')[0].data
# binterp_vec = np.arange(10,252.5,5)
# vinterp_vec = np.arange(-1000, 1050, 100)

In [6]:
def lnlike(theta, truth = lya_real):
    lya_model = model(theta)
    return -0.5 * np.nansum((lya_real - lya_model)**2), lya_model

In [7]:
theta = 585, -311, 0.2, 0.5, 0.1, 0.47 #vout, vin, aout, ain, gout, gin
likelihood, lya_model = lnlike(theta)
print(likelihood)
vmi, vma = -0.07, 0.97

fig, ax = plt.subplots(1,3, figsize = (14,6), sharex=True, sharey=True)
re = ax[0].imshow(lya_real, aspect='auto', origin='lower', 
                  extent = [bvec_final.min(), bvec_final.max(), vvec_final.min(), vvec_final.max()], vmin = vmi, vmax = vma)
fig.colorbar(re, ax=ax[0])
ax[0].set_title('Lya Real')

mod = ax[1].imshow(lya_model, aspect='auto', origin='lower', 
                   extent = [bvec_final.min(), bvec_final.max(), vvec_final.min(), vvec_final.max()], vmin = vmi, vmax = vma)
fig.colorbar(mod, ax=ax[1])
ax[1].set_title('Lya Model')

diff = ax[2].imshow(lya_real - lya_model, aspect='auto', origin='lower',
                    extent = [bvec_final.min(), bvec_final.max(), vvec_final.min(), vvec_final.max()], cmap='bwr', vmin=-0.3, vmax=0.3)
fig.colorbar(diff, ax=ax[2])
ax[2].set_title('Lya Real - Lya Model')
fig.tight_layout()
plt.show()

1024000000000000.0
10000.0


/var/folders/0j/64_lpts13_j1p6qcdv2njz8c0000gn/T/ipykernel_20024/57220483.py:6: RuntimeWarning: invalid value encountered in sqrt
  vout = np.sqrt(vi**2 + A*(-np.log((Rs+1)/Rs) + (1/r * np.log((Rs+r)/Rs))))
/var/folders/0j/64_lpts13_j1p6qcdv2njz8c0000gn/T/ipykernel_20024/3504095548.py:119: RuntimeWarning: Mean of empty slice
  tau_tot_in = np.nan_to_num(np.nanmean(tau_in_near)) + np.nan_to_num(np.nanmean(tau_in_far))
/var/folders/0j/64_lpts13_j1p6qcdv2njz8c0000gn/T/ipykernel_20024/3504095548.py:99: RuntimeWarning: Mean of empty slice
  tau_tot_out = np.nanmean(tau_out_near) + np.nan_to_num(np.nanmean(tau_out_far))


-6.6903036043267585
